In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn.preprocessing import Normalizer
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("working_csv.csv")
cols = df.columns
col_list = ["P. Habitable Class","P. Teq Max (K)","S. Mag from Planet","P. SFlux Mean (EU)","S. Teff (K)","P. Mag","P. HZD","P. ESI","P. SPH","P. HZC","P. Gravity (EU)","S. Size from Planet (deg)","S. [Fe/H]","S. DEC (deg)","P. SFlux Min (EU)","S. Hab Zone Max (AU)","P. HZI","P. Eccentricity","P. Appar Size (deg)"]
for words in cols:
    if words not in col_list:
        df.drop(columns = [words] , inplace = True)

In [3]:
df = df.sample(frac=1)

In [4]:
y = df["P. Habitable Class"]
X_train, X_test, Y_train, Y_test = train_test_split(df, y, test_size=0.2)

print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

(43, 19) (43,)
(11, 19) (11,)


In [5]:
dataset_train = X_train.values
X_train = dataset_train[:,1:19].astype(float)
Y_train = dataset_train[:,0]
dataset_test = X_test.values
X_test = dataset_test[:,1:19].astype(float)
Y_test = dataset_test[:,0] 
print(X_train)
print(Y_train)
print(X_test)
print(Y_test)

[[ 1.39000000e+00  1.46133200e+00  1.46133200e+00  2.80100000e+02
  -1.83200000e+01  2.66000000e+00  0.00000000e+00  3.49700000e+03
  -6.02714085e-02 -9.61240000e+00 -2.71000000e+01  1.75880000e+00
   3.11000000e-01 -9.90000000e-01 -1.60000000e-01  5.00000000e-01
   6.00000000e-02  8.20000000e-01]
 [ 2.07000000e+00  9.02172800e-01  9.02172900e-01  2.48300000e+02
  -1.83200000e+01  3.38000000e+00  0.00000000e+00  4.88600000e+03
  -6.02714085e-02  4.92125000e+01 -2.66000000e+01  7.07900000e-01
   1.01200000e+00 -5.30000000e-01 -1.40000000e-01  5.50000000e-01
   8.50000000e-01  8.00000000e-01]
 [ 2.24000000e+00  7.51445800e-01  1.43441000e+00  3.38000000e+02
  -1.89900000e+01  3.54000000e+00  3.40000000e-01  4.13400000e+03
  -3.00000000e-01  4.14522000e+01 -2.72000000e+01  1.28570000e+00
   5.71000000e-01 -8.90000000e-01 -1.40000000e-01  4.80000000e-01
   1.60000000e-01  7.50000000e-01]
 [ 1.52000000e+00  1.21709400e+02  1.21709400e+02  8.46200000e+02
  -2.32600000e+01  2.81000000e+00  0.

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y1 = np_utils.to_categorical(encoded_Y)
#print(dummy_y1)

encoder2 = LabelEncoder()
encoder2.fit(Y_test)
encoded_Y2 = encoder2.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y2 = np_utils.to_categorical(encoded_Y2)
print(dummy_y2)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [7]:
#scaler = MinMaxScaler()
#print(scaler.fit(X_train))
#print(scaler.data_max_)
#X_train = scaler.transform(X_train)

X_train = stats.zscore(X_train, axis=1, ddof=1)

#X_train = np.log(X_train + 1)
#X_train = normalize(X_train)

#transformer = Normalizer().fit(X_train)
#X_train = transformer.transform(X_train)

In [8]:
def accuracy(predictions, labels):
    preds_correct_boolean =  np.argmax(predictions, 1) == np.argmax(labels, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = 100.0 * correct_predictions / predictions.shape[0]
    return accuracy

In [9]:
def relu_activation(data_array):
    return np.maximum(data_array, 0)

In [10]:
def softmax(output_array):
    logits_exp = np.exp(output_array.astype(np.float32))
    return logits_exp / np.sum(logits_exp, axis = 1, keepdims = True)

In [11]:
def cross_entropy_softmax_loss_array(softmax_probs_array, y_onehot):
    indices = np.argmax(y_onehot, axis = 1).astype(int)
    predicted_probability = softmax_probs_array[np.arange(len(softmax_probs_array)), indices]
    log_preds = np.log(predicted_probability)
    loss = -1.0 * np.sum(log_preds) / len(log_preds)
    return loss

In [12]:
def regularization_L2_softmax_loss(reg_lambda, weight1, weight2):
    weight1_loss = 0.5 * reg_lambda * np.sum(weight1 * weight1)
    weight2_loss = 0.5 * reg_lambda * np.sum(weight2 * weight2)
    return weight1_loss + weight2_loss

In [13]:
data = X_train
labels = dummy_y1

hidden_nodes = 6
num_labels = labels.shape[1]
num_features = data.shape[1]
learning_rate = .01
reg_lambda = .01

# Weights and Bias Arrays
layer1_weights_array = np.random.normal(0, 1, [num_features, hidden_nodes]) 
layer2_weights_array = np.random.normal(0, 1, [hidden_nodes, num_labels]) 

layer1_biases_array = np.zeros((1, hidden_nodes))
layer2_biases_array = np.zeros((1, num_labels))


for step in range(5501):

    input_layer = np.dot(data, layer1_weights_array)
    hidden_layer = relu_activation(input_layer + layer1_biases_array)
    output_layer = np.dot(hidden_layer, layer2_weights_array) + layer2_biases_array
    output_probs = softmax(output_layer)
    
    loss = cross_entropy_softmax_loss_array(output_probs, labels)
    loss += regularization_L2_softmax_loss(reg_lambda, layer1_weights_array, layer2_weights_array)

    output_error_signal = (output_probs - labels) / output_probs.shape[0]
    
    error_signal_hidden = np.dot(output_error_signal, layer2_weights_array.T) 
    error_signal_hidden[hidden_layer <= 0] = 0
    
    gradient_layer2_weights = np.dot(hidden_layer.T, output_error_signal)
    gradient_layer2_bias = np.sum(output_error_signal, axis = 0, keepdims = True)
    
    gradient_layer1_weights = np.dot(data.T, error_signal_hidden)
    gradient_layer1_bias = np.sum(error_signal_hidden, axis = 0, keepdims = True)

    gradient_layer2_weights += reg_lambda * layer2_weights_array
    gradient_layer1_weights += reg_lambda * layer1_weights_array

    layer1_weights_array -= learning_rate * gradient_layer1_weights
    layer1_biases_array -= learning_rate * gradient_layer1_bias
    layer2_weights_array -= learning_rate * gradient_layer2_weights
    layer2_biases_array -= learning_rate * gradient_layer2_bias
    
    if step % 100 == 0:
            print("Loss at step " , step , " : " , loss)

Loss at step  0  :  2.0240180660930505
Loss at step  100  :  1.6119641665321573
Loss at step  200  :  1.5867972130284587
Loss at step  300  :  1.5640076624589774
Loss at step  400  :  1.5428644643051168
Loss at step  500  :  1.5227114885046893
Loss at step  600  :  1.5019646603842225
Loss at step  700  :  1.4836111274982753
Loss at step  800  :  1.4663074198524526
Loss at step  900  :  1.448559126384389
Loss at step  1000  :  1.4330010877537265
Loss at step  1100  :  1.4183451311479534
Loss at step  1200  :  1.4044977801036373
Loss at step  1300  :  1.3913659883059766
Loss at step  1400  :  1.3788640474375438
Loss at step  1500  :  1.3669176960941154
Loss at step  1600  :  1.3553221832955615
Loss at step  1700  :  1.344203087299993
Loss at step  1800  :  1.333547500304333
Loss at step  1900  :  1.323259745617884
Loss at step  2000  :  1.3133384384252031
Loss at step  2100  :  1.3037507780778204
Loss at step  2200  :  1.2944915209260515
Loss at step  2300  :  1.285538048622122
Loss at s

In [14]:
input_layer = np.dot(X_test, layer1_weights_array)
hidden_layer = relu_activation(input_layer + layer1_biases_array)
scores = np.dot(hidden_layer, layer2_weights_array) + layer2_biases_array
probs = softmax(scores)
print(accuracy(probs, dummy_y2))

36.36363636363637


/home/chirag/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/chirag/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
print(probs)

[[ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]
 [ 0. nan  0.]]
